In [3]:
import requests

In [4]:
weights_url = "https://pjreddie.com/media/files/yolov3.weights"
cfg_url = "https://github.com/pjreddie/darknet/blob/master/cfg/yolov3.cfg?raw=true"
names_url = "https://github.com/pjreddie/darknet/blob/master/data/coco.names?raw=true"

In [5]:
def download_file(url, filename):
  with open(filename, 'wb') as file:
    response = requests.get(url, stream=True)
    file.write(response.content)

In [6]:
download_file(weights_url, 'yolov3.weights')
download_file(cfg_url, 'yolov3.cfg')
download_file(names_url, 'coco.names')

In [1]:
import cv2
import numpy as np

In [7]:
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")

In [9]:
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers().tolist()]

In [10]:
with open("coco.names", "r") as f:
  classes = [line.strip() for line in f.readlines()]
image_path = "objects.jpg"
image = cv2.imread(image_path)
height, width, channels = image.shape

In [14]:
from google.colab.patches import cv2_imshow

In [ ]:
blob = cv2.dnn.blobFromImage(image, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
net.setInput(blob)
outs = net.forward(output_layers)
for out in outs:
  for detection in out:
    print(1)
    scores = detection[5:]
    class_id = np.argmax(scores)
    confidence = scores[class_id]
    if confidence > 0.5:
      center_x, center_y, w, h = (detection[0:4] * np.array([width, height, width, height])).astype('int')
      x = int(center_x - w / 2)
      y = int(center_y - h / 2)
      cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
      cv2.putText(image, f"{classes[class_id]}{int(confidence * 100)}%", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
cv2_imshow(image)
cv2.waitKey(0)
cv2.destroyAllWindows()